In [ ]:
import os
import h5py
import numpy as np
from typing import List, Dict, Callable
from collections import defaultdict

from ml4h.arguments import _get_tmap
from ml4h.TensorMap import TensorMap

In [ ]:
def tmaps_by_sample_id(tensor_folder: str, sample_id: str, tmaps: List[TensorMap]):
    path = os.path.join(tensor_folder, sample_id + '.hd5')
    result_dict = defaultdict(lambda: None)
    if os.path.isfile(path):
            with h5py.File(path, 'r') as hd5:
                for tmap in tmaps:
                    try:
                        result_dict[tmap] = tmap.tensor_from_file(tmap, hd5)
                    except (IndexError, KeyError, ValueError, OSError, RuntimeError):
                        continue
    return result_dict


def tmaps_with_properties(tensor_folder: str, tmap_properties: Dict[TensorMap, Callable[[np.ndarray], bool]], search_size=100):
    all_ids = [file.strip('.hd5') for file in sorted(os.listdir(tensor_folder))[:search_size]]
    results = map(lambda sample_id: tmaps_by_sample_id(tensor_folder, sample_id, tmap_properties.keys()), all_ids)
    return {
        sample_id: result
        for sample_id, result in zip(all_ids, results)
        if all(
            result[tmap] is not None and tmap_properties[tmap](result[tmap])
            for tmap in tmap_properties.keys()
        )
    }


def tmaps_with_properties_from_keys(tensor_folder: str, tmap_properties: Dict[str, Callable[[np.ndarray], bool]], search_size=100):
    return tmaps_with_properties(
        tensor_folder,
        {_get_tmap(key): prop for key, prop in tmap_properties.items()},
        search_size,
    )

# Examples

In [ ]:
tmaps_by_sample_id('/mnt/disks/ecg-bike-tensors/2019-10-10/', '1003126', [_get_tmap('ecg-bike-resting-hr')])

In [ ]:
tmaps_with_properties(
    '/mnt/disks/ecg-bike-tensors/2019-10-10/', 
    {
        _get_tmap('ecg-bike-resting-hr'): lambda x: x < -2
    }
)

In [ ]:
tmaps_with_properties_from_keys(
    '/mnt/disks/ecg-bike-tensors/2019-10-10/', 
    {
        'ecg-bike-resting-hr': lambda x: x < -2
    }
)